# Training Pipeline

In this notebook, we will do the following tasks

1. Get the data from the feature store
2. Preprocess the data
3. Train the model
4. Evaluate the model
5. Register the model to model registry


In [1]:
# Import the required libraries
import os
import hopsworks
import numpy as np
import pandas as pd

from dotenv import load_dotenv

# Load the .env file 
load_dotenv()

# Get the envrioment variables
hopsworks_api_key = os.getenv("HOPSWORKS_API_KEY")
 

In [2]:
# login hopsworks and get the features group
project = hopsworks.login(api_key_value=str(hopsworks_api_key))
fs = project.get_feature_store()



2025-02-22 12:49:20,238 INFO: Initializing external client
2025-02-22 12:49:20,238 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 12:49:24,169 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212597


## Feature View


### Feature Selection


In [3]:
amazon_fg = fs.get_feature_group("amazon_stock_prices", version=1)

In [4]:
# Select features for training data
selected_features = amazon_fg.select(["date", "open", "high", "close", "low"])

# View the first 5 rows of selected features
selected_features.show(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.56s) 


,date,open,high,close,low
0,2024-03-19 17:00:00+00:00,175.710,175.985,175.540,175.33
1,2024-05-24 14:00:00+00:00,181.570,182.165,181.350,181.04
2,2023-06-14 16:00:00+00:00,126.450,126.460,126.305,126.07
3,2023-02-14 16:00:00+00:00,98.470,98.855,97.970,97.53
4,2023-07-17 14:00:00+00:00,134.075,134.710,134.065,133.61


### Feature View Creation


In [5]:
# Get or create feature view
amazon_fv = fs.get_or_create_feature_view(
    name= "amazon_fv", 
    version=1,
    query = selected_features,
)

## Training Dataset Creation


In [6]:
# Get the data to calculate date ranges
df = amazon_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.76s) 


In [7]:
# Sort the values according to the date
df = df.sort_values("date").set_index("date")

df.head()

,close,high,low,open,id
date,,,,,
2022-06-07 14:00:00+00:00,123.350,124.090,121.55,122.440,2022-06-07 14:00:00
2022-06-07 15:00:00+00:00,122.420,123.375,121.53,123.360,2022-06-07 15:00:00
2022-06-07 16:00:00+00:00,121.670,122.500,121.03,122.500,2022-06-07 16:00:00
2022-06-07 17:00:00+00:00,122.085,122.475,121.40,121.680,2022-06-07 17:00:00
2022-06-07 18:00:00+00:00,122.190,122.810,121.95,122.085,2022-06-07 18:00:00


In [8]:
def get_fractional_dates(column: pd.Series):
    total_length = len(column)
    
    idx_0 = 0  # First index
    idx_70 = int(0.7 * total_length)
    idx_85 = int(0.85 * total_length)
    idx_last = total_length - 1  # Last index

    def extract_date(idx):
        return str(column.index[idx]).split(" ")[0]  # Convert to string and get date part

    train_start = extract_date(idx_0)
    val_start = extract_date(idx_70)
    test_start = extract_date(idx_85)

    train_end = extract_date(idx_70 - 1)  # One day before val_start
    val_end = extract_date(idx_85 - 1)  # One day before test_start
    test_end = extract_date(idx_last)  # Last available date

    return train_start, train_end, val_start, val_end, test_start, test_end

In [9]:
train_start, train_end, val_start, val_end, test_start, test_end = get_fractional_dates(df)
train_start, train_end, val_start, val_end, test_start, test_end

('2022-06-07',
 '2024-04-30',
 '2024-04-30',
 '2024-09-25',
 '2024-09-25',
 '2025-02-21')

#### Split the data into train, val and test splits


In [10]:
train, val, test, _, _, _ =  amazon_fv.train_validation_test_split(
    train_start=train_start,
    train_end = train_end,
    val_start = val_start,
    val_end = val_end,
    test_start=test_start,
    test_end = test_end    
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.49s) 
2025-02-22 12:49:39,945 WARNING: VersionWarning: Incremented version to `12`.



In [ ]:
train['date'] = pd.to_datetime(train['date'], utc=True)
val['date'] = pd.to_datetime(val['date'], utc=True)
test['date'] = pd.to_datetime(test['date'], utc=True)

In [24]:
# Sort and remove the date column to ensure all data is in the same format
train = train.sort_values("date").drop("date", axis=1)
val = val.sort_values("date").drop("date", axis=1)
test = test.sort_values("date").drop("date", axis=1)

### Creating TorchDataset and DataLoaders


In [25]:
import torch
import comet_ml
from torch import nn
from torch.utils.data import DataLoader, Dataset


# Setup hyperparameters
hyper_params = {
    'input_size' : 4,  # Number of features
    "window_size": 24,
    "forecast_steps" : 6,
    "batch_size" : 32,
    "hidden_size" : 128,
    "num_layers" : 4,
    "num_epochs" : 100,
    "learning_rate": 0.001
}

In [26]:
def create_sequences(data: pd.DataFrame, window_size: int = hyper_params['window_size'], forecast_steps: int = hyper_params['forecast_steps']):
    # Initialize empty lists
    X, y = [], []
        
    # Loop through the dataset
    for i in range(len(data) - window_size - forecast_steps):
        X.append(data.iloc[i: i + window_size, :])
        
        # y values should start from the next value where X ended to forecast steps
        # close is the target column and is at the 3rd place
        y.append(data.iloc[i + window_size: i + window_size + forecast_steps, 2])
        
    return np.array(X), np.array(y)

In [27]:
# Test our data
X_train, y_train  = create_sequences(train)
X_val, y_val = create_sequences(val)
X_test, y_test = create_sequences(test)

In [28]:
# View the shapes of train, val and test splits
print(f"Shape of Training Data: {X_train.shape, y_train.shape}")
print(f"Shape of Validation Data: {X_val.shape, y_val.shape}")
print(f"Shape of Testing Data: {X_test.shape, y_test.shape}")

Shape of Training Data: ((2934, 24, 4), (2934, 6))
Shape of Validation Data: ((606, 24, 4), (606, 6))
Shape of Testing Data: ((606, 24, 4), (606, 6))


In [29]:
# Create Torch Dataset

class AmazonDataset(Dataset):
    """ Creates Amazon Stock Price Dataset """
    def __init__(self, X: np.ndarray,y: np.ndarray):
        super().__init__()  # Call the parent class constructor
        self.X = X
        self.y = y
        
    def __len__(self):
            return len(self.X)
        
    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)

In [30]:
# Create dataset and dataloaders
train_data = AmazonDataset(X_train, y_train)
val_data = AmazonDataset(X_val, y_val)
test_data = AmazonDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=hyper_params['batch_size'], shuffle=True)
val_loader = DataLoader(val_data, batch_size=hyper_params['batch_size'], shuffle=False)
test_loader = DataLoader(test_data, batch_size=hyper_params['batch_size'], shuffle=False)


In [31]:
len(train_loader), len(val_loader), len(test_loader)

(92, 19, 19)

In [32]:
# check the shape of single batch

for X, y in train_loader:
    print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")
    break

Shape of X: torch.Size([32, 24, 4]), Shape of y: torch.Size([32, 6])


## Modelling


In [33]:
# Setup Logging in Comet ml
from comet_ml.integration.pytorch import watch

# Set up the Comet experiment
load_dotenv()
comet_api_key = os.getenv("COMET_API_KEY")

# Login to comret
comet_ml.login(api_key=comet_api_key)



experiment = comet_ml.start(project_name='lstm-stock-price-prediction')
experiment.log_parameters(hyper_params)


COMET INFO: Valid Comet API Key saved in C:\Users\amuly\.comet.config (set COMET_CONFIG to change where it is saved).
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/amulyaprasanth/lstm-stock-price-prediction/30ae5d089fee407da8f730eb449a57dc



2025-02-22 12:54:15,604 WARNING: DeprecationWarning: Please use dulwich.object_store.iter_tree_contents



In [34]:
# Create the model
class LSTMModel(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int, num_layers: int, device:str = 'cpu'):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.device = device
        
		# LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
	# forward pass
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(self.device)
        
        out, (_, _) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
        


In [35]:
from torchinfo import summary

# Get the summary of the model
summary(LSTMModel(input_dim=X_train.shape[2], hidden_dim=hyper_params['hidden_size'], output_dim=6, num_layers=2, device='cuda'), input_size=(hyper_params['batch_size'], X_train.shape[1], X_train.shape[2]))

Layer (type:depth-idx)                   Output Shape              Param #
LSTMModel                                [32, 6]                   --
├─LSTM: 1-1                              [32, 24, 128]             200,704
├─Linear: 1-2                            [32, 6]                   774
Total params: 201,478
Trainable params: 201,478
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 154.17
Input size (MB): 0.01
Forward/backward pass size (MB): 0.79
Params size (MB): 0.81
Estimated Total Size (MB): 1.61

In [36]:
from tqdm.auto import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader

def train_model(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, num_epochs: int, device: str = 'cuda'):
    """
    Trains and validates a PyTorch model.

    Args:
        model (nn.Module): The neural network model to be trained.
        train_loader (DataLoader): DataLoader for the training data.
        val_loader (DataLoader): DataLoader for the validation data.
        loss_fn (nn.Module): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer.
        num_epochs (int): Number of epochs to train the model.
        device (str): Device to run the model on ('cuda' or 'cpu').

    Returns:
        None
    """
    # Move model to GPU if available
    model = model.to(device)

    watch(model)
    for epoch in tqdm(range(num_epochs), desc= 'Epochs: '):
        train_loss, test_loss = 0.0, 0.0
        
        ### Train time
        # Set the model to training mode
        model.train()
        
        # Iterate over the training data
        for X, y in tqdm(train_loader, desc='Training...', leave=False):
            # Move data to GPU if available
            X, y = X.to(device), y.to(device)

            # optimizer zero grad
            optimizer.zero_grad()
            
            # Do the forward pass
            outputs = model(X)
            
            # Calculate the loss
            loss = loss_fn(outputs, y)
            train_loss += loss.item()
            
            
            # loss backward
            loss.backward()
            
            # optimizer step
            optimizer.step()
    
        ### Test time
        # Set the model to evaluation mode
        model.eval()
        
        with torch.inference_mode():
            # Iterate over the validation data
            for X, y in tqdm(val_loader, desc='Evaluating...', leave=False):
                # Move data to GPU if available
                X, y = X.to(device), y.to(device)
                
                # Do the forward pass
                outputs = model(X)
                
                # Calculate the loss
                loss = loss_fn(outputs, y)
                test_loss += loss.item()

        train_loss /= len(train_loader)
        test_loss /= len(val_loader)

        # Log the train and test loss to Comet ML
        experiment.log_metric("train_loss", train_loss, epoch=epoch)
        experiment.log_metric("test_loss", test_loss, epoch=epoch)
        
        # Print the train and test loss
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:7f}, Test Loss: {test_loss:.7f}")

    # End the experiment
    experiment.end()

In [37]:
# Define the loss function and optimizer
model = LSTMModel(input_dim=hyper_params['input_size'], hidden_dim=hyper_params['hidden_size'], output_dim=hyper_params['forecast_steps'], num_layers=hyper_params['num_layers'], device='cuda')
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate'])

# Train the model with logging
train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=hyper_params['num_epochs'], device='cuda')

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Training...:   0%|          | 0/92 [00:00<?, ?it/s]

Evaluating...:   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 1/100, Train Loss: 14946.181577, Test Loss: 28417.9646382


Training...:   0%|          | 0/92 [00:00<?, ?it/s]

...

Training...:   0%|          | 0/92 [00:00<?, ?it/s]

Evaluating...:   0%|          | 0/19 [00:00<?, ?it/s]

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : striking_pigeon_3947
COMET INFO:     url                   : https://www.comet.com/amulyaprasanth/lstm-stock-price-prediction/30ae5d089fee407da8f730eb449a57dc
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_loss [100]  : (61.218498066851964, 28417.964638157893)
COMET INFO:     train_loss [100] : (7.266649453536324, 14946.181576936142)
COMET INFO:   Parameters:
COMET INFO:     batch_size     : 32
COMET INFO:     forecast_steps : 6
COMET INFO:     hidden_size    : 128
COMET INFO:     input_size     : 4
COMET INFO:     learning_rate  : 0.001
COMET INFO:     num_epochs     : 100
COMET INFO:     num_layers     : 4
COMET INFO:     window_siz

Epoch 100/100, Train Loss: 7.965576, Test Loss: 61.2184981


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET INFO: Please wait for metadata to finish uploading (timeout is 3600 seconds)
COMET INFO: Uploading 1 metrics, params and output messages


In [38]:
# Evalute the model on the test set
test_preds = []
model.eval()
with torch.inference_mode():
    for X, y in test_loader:
        X, y = X.to('cuda'), y.to('cuda')
        outputs = model(X)
        test_preds.extend(outputs.cpu().numpy())

In [39]:
from sklearn.metrics import mean_squared_error

# Calculate the mean squared error

mse = mean_squared_error(y_test, test_preds)
mse

1409.0451315421453

## Register Model

In [40]:
# Create model direcotry if it doesn't exist
from hsml.model_schema import ModelSchema
from hsml.schema import Schema
import json
import joblib
input_schema = Schema(X_train)
output_schema = Schema(y_train)

# Get a random example from the numpy array
random_index = np.random.randint(X_train.shape[0])
input_example = X_train[random_index, :]

model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema, input_example=input_example)

# Create the model directory
model_dir = '../models/lstm-stock-price-prediction'
hyper_params_file_path = os.path.join("../preprocessor/", 'hyper_params.json')
os.makedirs('../preprocessor', exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

# Save the model and preprocessor object
model_file_path = os.path.join(model_dir, 'model.pt')

# Save the preprocessor and hyperparameters
with open(hyper_params_file_path, 'w') as f:
    json.dump(hyper_params, f)

torch.save(model.state_dict(), model_file_path)

### Register model in registry


In [41]:
metrics = {'mean_squared_error': mse}

mr = project.get_model_registry()

lstm_torch_model = mr.torch.create_model("amazon_stock_price_prediction_model_torch", metrics=metrics)
lstm_torch_model.save(model_file_path)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1865606 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1212597/models/amazon_stock_price_prediction_model_torch/2


Model(name: 'amazon_stock_price_prediction_model_torch', version: 2)